# FBA Problem - Group 5 

Study the metabolism of ***C. reinhardtii*** under different conditions, aiming at predicting and improving environmental and genetic conditions to **maximize the production** of one desirable compound.


### Metabolites:

<left><img src='metabolitos.png' width='400'></left>

### Conditions:

<left><img src='cndicoes.png' width='400'></left>

### BPCY Formulation

$ BPCY = \frac{Product \cdot Growth}{Substrate}  $




In [25]:
# Importação de bibliotecas e módulos para resolução de exercícios

from cobra.io import read_sbml_model
from mewpy.simulation import get_simulator
import cobra
import pandas as pd

from mewpy.omics import ExpressionSet
from mewpy.omics import eFlux    
from mewpy.omics import GIMME         

In [56]:
# Carregamos o modelo usando 

modelo = cobra.io.read_sbml_model('iRC1080_lv3.xml')

# Exercise 2

In stress conditions, microalgae  often change  their  metabolism to fight the alterations in the environment. In these scenarios, some carbon can be secreted in the 
form  of  organic acids. Although  some  stress  conditions  can  be  replicated directly  in GSM  models,  others, such as  temperature,  pH,  salinity,  require  the addition of additional  information,  such  as  gene expression  data. The file “expression_data.tsv” contains  the  normalized  gene  expression  profile  of  C.  reinhardii in  two  conditions: control and stress. 

## 2.a. 

Integrate the  expression  data for  both  conditions  using  the  eFLUX  algorithm. 
If your metabolite is H2, use the column “Stress_h2” instead of “Stress”.  The environmental condition  for each group/metabolite is available in Table  1.


In [43]:
'''
Definimos as condições ambientais. 
No caso do nosso grupo as condições são 'microaerobic mixotrophic'

photon -2000
co2 -11.6
o2 -0.01
acetate -10
starch -1.72e-04

'''
modelo.objective = 'BIOMASS_Chlamy_mixo'
modelo = modelo.optimize
cond_ambientais = {
        'EX_photonVis_e':(-2000    , 1000),   # condições de luz
        'EX_co2_e'      :(-11.6    , 1000),   # consumo C02
        'EX_o2_e'       :(-0.01    , 1000),   # condições de microaerobiose
        'EX_ac_e'       :(-10      , 1000),   # consumo acetato
        'SK_starch300_h':(-1.72e-4 , 1000),   # consumo amido
                    
          }

In [49]:
# Carregamento do ficheiro de dados de expressão genética

exp_gen = pd.read_csv('expression_data.tsv',sep='\t',index_col=0)

# Como não nos interessa 'Stress_12' vamos eliminar a coluna

exp_gen.drop('Stress_12h', axis = 1, inplace = True)

In [50]:
exp_gen.columns

Index(['Control_12h', 'Stress_12_h2'], dtype='object')

In [51]:
# Estabelecemos as condições para a simulação usando MEWpy
simulador = get_simulator(modelo, envcond = cond_ambientais)

# Tratamos os dados para integração

genes      = exp_gen.index.values                     # Isolamos a lista de genes
ccontrolo  = ['Control_12h' ]  # Definimos as nossas condições
cstress    = ['Stress_12_h2']
controlo   = exp_gen[ccontrolo].to_numpy()         # Definimos os dados de expressão para cada uma das condições - controlo
stress     = exp_gen[cstress  ].to_numpy()         # Definimos os dados de expressão para cada uma das condições - stress

# Criamos o módulo de expressão

exp_controlo = ExpressionSet(genes, ccontrolo, controlo)
exp_stress   = ExpressionSet(genes, cstress  , controlo)

ValueError: The model <builtins.method> has no defined simulator.

In [52]:
# Realizamos a integração dos dados para a condição controlo
# Usamos o algoritmo eFLUX

g_c = eFlux(simulador,exp_controlo)


In [53]:
g_c.find(['H2td','H2th','Hts','EX_h2_e'])

,Flux rate
Reaction ID,
Hts,5.122614e-21
NADHts,-5.122614e-21
NADPHts,5.122614e-21


In [54]:
# Realizamos a integração dos dados para a condição de stress
# Usamos o algoritmo eFLUX

g_s = eFlux(simulador,exp_stress)

In [55]:
g_s.find(['H2td','H2th','Hts','EX_h2_e'])

,Flux rate
Reaction ID,
Hts,5.122614e-21
NADHts,-5.122614e-21
NADPHts,5.122614e-21


## 2.b

# Exercise 3

## 3.a

In [35]:
# Genes essenciais

genes_ess = cobra.flux_analysis.find_essential_genes(modelo)

In [36]:
len(genes_ess)

136

In [37]:
# Reacções essenciais
reacts_ess = cobra.flux_analysis.find_essential_reactions(modelo)

In [38]:
len(reacts_ess)

336

## 3.b